# Car Detection

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/car_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [3]:
pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.4 MB/s eta 0:00:00
  

## Import libraries

In [1]:
import geoai

## Download sample data

We will download a sample image from Hugging Face Hub to use for car detection. You can find more high-resolution images from [OpenAerialMap](https://openaerialmap.org).

In [2]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_7cm.tif"
)

In [3]:
raster_path = geoai.download_file(raster_url)

cars_7cm.tif: 100%|██████████| 92.0M/92.0M [00:01<00:00, 79.9MB/s]


## Visualize the image

In [4]:
geoai.view_raster(raster_url)

## Initialize the model

In [6]:
detector = geoai.CarDetector()

Model path not specified, downloading from Hugging Face...
Error downloading model from Hugging Face: 404 Client Error. (Request ID: Root=1-67db0b64-025d08032c03f71369f15481;fff988b7-154a-47a2-81b4-8fe66a414e8e)

Entry Not Found for url: https://huggingface.co/giswqs/geoai/resolve/main/car_detection_usa.pth.
Please specify a local model path or ensure internet connectivity.


EntryNotFoundError: 404 Client Error. (Request ID: Root=1-67db0b64-025d08032c03f71369f15481;fff988b7-154a-47a2-81b4-8fe66a414e8e)

Entry Not Found for url: https://huggingface.co/giswqs/geoai/resolve/main/car_detection_usa.pth.

## Extract cars

Extract cars from the image using the model and save the output image.

In [ ]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="cars_masks.tif",
    confidence_threshold=0.3,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(400, 400),
)

Convert the image masks to polygons and save the output GeoJSON file.

In [ ]:
gdf = detector.vectorize_masks(
    masks_path="cars_masks.tif",
    output_path="cars.geojson",
    min_object_area=100,
    max_object_area=2000,
)

## Add geometric properties

In [ ]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize initial results

In [ ]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter cars by area

In [ ]:
gdf_filter = gdf[
    (gdf["area_m2"] > 8) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [ ]:
len(gdf_filter)

## Visualiza final results

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

In [ ]:
geoai.view_vector_interactive(gdf_filter, tiles=raster_url)

![image](https://github.com/user-attachments/assets/a1e4c871-b152-466a-b902-7c00e62e5444)